# Question Answering with LangChain, OpenAI, and MultiQuery Retriever

This interactive workbook demonstrates example of Elasticsearch's [MultiQuery Retriever](https://api.python.langchain.com/en/latest/retrievers/langchain.retrievers.multi_query.MultiQueryRetriever.html) to generate similar queries for a given user input and apply all queries to retrieve a larger set of relevant documents from a vectorstore.

Before we begin, we first split the fictional workplace documents into passages with `langchain` and uses OpenAI to transform these passages into embeddings and then store these into Elasticsearch.

We will then ask a question, generate similar questions using langchain and OpenAI, retrieve relevant passages from the vector store, and use langchain and OpenAI again to provide a summary for the questions.

## Install packages and import modules

In [3]:
#!pip install -qU jq lark langchain langchain-elasticsearch langchain_openai tiktoken

from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_elasticsearch import ElasticsearchStore
from langchain_openai.llms import OpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever
from getpass import getpass

## Connect to Elasticsearch

ℹ️ We're using an Elastic Cloud deployment of Elasticsearch for this notebook. If you don't have an Elastic Cloud deployment, sign up [here](https://cloud.elastic.co/registration?utm_source=github&utm_content=elasticsearch-labs-notebook) for a free trial. 

We'll use the **Cloud ID** to identify our deployment, because we are using Elastic Cloud deployment. To find the Cloud ID for your deployment, go to https://cloud.elastic.co/deployments and select your deployment.

We will use [ElasticsearchStore](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.elasticsearch.ElasticsearchStore.html) to connect to our elastic cloud deployment, This would help create and index data easily.  We would also send list of documents that we created in the previous step

In [5]:
# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#finding-your-cloud-id
ELASTIC_CLOUD_ID = getpass("Elastic Cloud ID: ")

# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#creating-an-api-key
ELASTIC_API_KEY = getpass("Elastic Api Key: ")

# https://platform.openai.com/api-keys
OPENAI_API_KEY = getpass("OpenAI API key: ")

embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

vectorstore = ElasticsearchStore(
    es_cloud_id=ELASTIC_CLOUD_ID,
    es_api_key=ELASTIC_API_KEY,
    index_name="multi-query-lab",
    embedding=embeddings,
)

## Indexing Data into Elasticsearch
Let's download the sample dataset and deserialize the document.

In [6]:
from urllib.request import urlopen
import json

url = "https://raw.githubusercontent.com/elastic/elasticsearch-labs/main/example-apps/chatbot-rag-app/data/data.json"

response = urlopen(url)
data = json.load(response)

with open("temp.json", "w") as json_file:
    json.dump(data, json_file)

### Split Documents into Passages

We’ll chunk documents into passages in order to improve the retrieval specificity and to ensure that we can provide multiple passages within the context window of the final question answering prompt.

Here we are chunking documents into 800 token passages with an overlap of 400 tokens.

Here we are using a simple splitter but Langchain offers more advanced splitters to reduce the chance of context being lost.

In [8]:
from langchain.document_loaders import JSONLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


def metadata_func(record: dict, metadata: dict) -> dict:
    #Populate the metadata dictionary with keys name, summary, url, category, and updated_at.
    metadata['name'] = record.get('name')
    metadata['summary'] = record.get('summary')
    metadata['url'] = record.get('url')
    metadata['category'] = record.get('category')
    metadata['updated_at'] = record.get('updated_at')

    return metadata


# For more loaders https://python.langchain.com/docs/modules/data_connection/document_loaders/
# And 3rd party loaders https://python.langchain.com/docs/modules/data_connection/document_loaders/#third-party-loaders
loader = JSONLoader(
    file_path="temp.json",
    jq_schema=".[]",
    content_key="content",
    metadata_func=metadata_func,
)

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=800, chunk_overlap=400 #define chunk size and chunk overlap
)
docs = loader.load_and_split(text_splitter=text_splitter)

### Bulk Import Passages

Now that we have split each document into the chunk size of 800, we will now index data to elasticsearch using [ElasticsearchStore.from_documents](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.elasticsearch.ElasticsearchStore.html#langchain.vectorstores.elasticsearch.ElasticsearchStore.from_documents).

We will use Cloud ID, Password and Index name values set in the `Create cloud deployment` step.

In [10]:
documents = vectorstore.from_documents(
    docs,
    embeddings,
    index_name="multi-query-lab",
    es_cloud_id=ELASTIC_CLOUD_ID,
    es_api_key=ELASTIC_API_KEY,
)

llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

retriever = MultiQueryRetriever.from_llm(vectorstore.as_retriever(), llm)

# Question Answering with MultiQuery Retriever

Now that we have the passages stored in Elasticsearch, we can now ask a question to get the relevant passages.

In [11]:
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.schema import format_document

import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

LLM_CONTEXT_PROMPT = ChatPromptTemplate.from_template(
    """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Be as verbose and educational in your response as possible. 
    
    context: {context}
    Question: "{question}"
    Answer:
    """
)

LLM_DOCUMENT_PROMPT = PromptTemplate.from_template(
    """
---
SOURCE: {name}
{page_content}
---
"""
)


def _combine_documents(
    docs, document_prompt=LLM_DOCUMENT_PROMPT, document_separator="\n\n"
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)


_context = RunnableParallel(
    context=retriever | _combine_documents,
    question=RunnablePassthrough(),
)

chain = _context | LLM_CONTEXT_PROMPT | llm

ans = chain.invoke("what is the nasa sales team?")

print("---- Answer ----")
print(ans)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. Can you provide information on the sales team at NASA?', '2. How does the sales team operate within NASA?', '3. What are the responsibilities of the NASA sales team?']


---- Answer ----
The NASA sales team is a part of the Americas region in the sales organization of the company. It is led by two Area Vice-Presidents, Laura Martinez for North America and Gary Johnson for South America. The team is responsible for promoting and selling the company's products and services in the North and South American markets. They work closely with other departments, such as marketing, product development, and customer support, to ensure the company's success in these regions.


**Generate at least two new iteratioins of the previous cells - Be creative.** Did you master Multi-
Query Retriever concepts through this lab?

In [12]:
ans2 = chain.invoke("explain to me what a home work policy is?")

print("---- Answer ----")
print(ans2)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. Can you provide me with information on the home work policy?', '2. What are the key components of a home work policy?', '3. How does a home work policy benefit employees and employers?']


---- Answer ----

A work-from-home policy is a set of guidelines and procedures that outline the expectations and requirements for employees who are allowed to work remotely from their homes. This policy is typically put in place to ensure the continuity and productivity of business operations, especially during times of crisis such as the COVID-19 pandemic. It covers various aspects such as eligibility, equipment and resources, communication, work hours and availability, performance expectations, time tracking and overtime, confidentiality and data security, health and well-being, and policy review and updates. The policy also encourages employees to prioritize their health and well-being while working from home and provides a process for addressing any questions or concerns.


In [13]:
ans3 = chain.invoke("what roles and responsibilities does a software engineer have?")

print("---- Answer ----")
print(ans3)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. What are the duties and obligations of a software engineer?', '2. Can you list the tasks and expectations of a software engineer?', '3. How would you describe the job responsibilities of a software engineer?']


---- Answer ----

A software engineer is responsible for designing, developing, and maintaining software applications and components. They collaborate with team members to write clean, efficient code following established coding standards. They also participate in code reviews, troubleshoot and resolve software defects and issues, and assist with the creation of technical documentation. Additionally, software engineers are expected to continuously learn and stay up-to-date with new technologies and best practices. The specific roles and responsibilities may vary depending on the level of experience and seniority, as outlined in the Swe Career Matrix. Junior software engineers may have more basic responsibilities, while senior and principal software engineers may have additional responsibilities such as leading and mentoring junior team members, driving the adoption of advanced software development practices and technologies, and representing the company as a thought leader in the softw

In [14]:
ans4 = chain.invoke("can you write me some onboarding policies for a intern at a consulting firm?")

print("---- Answer ----")
print(ans4)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. What are the onboarding policies for an intern at a consulting firm?', '2. Can you provide me with some guidelines for onboarding an intern at a consulting firm?', '3. How can a consulting firm effectively onboard their interns?']


---- Answer ----

Sure, here are some onboarding policies for an intern at a consulting firm:

1. Orientation: All interns will be required to attend an orientation session to learn about the company's history, mission, and values. This will also be an opportunity to meet other interns and team members.

2. Training: Interns will be provided with necessary training to familiarize themselves with the company's policies, procedures, and tools. This may include training on specific software or tools used by the company.

3. Mentorship: Each intern will be assigned a mentor who will provide guidance and support throughout their internship. The mentor will also be responsible for setting goals and providing feedback to help the intern grow and develop.

4. Project Assignments: Interns will be assigned to specific projects or tasks based on their skills and interests. They will work closely with their mentor and other team members to complete these assignments.

5. Performance Evaluations: I